# CL Protocol Specification

Code is used to show the mathematics details for ZKP-CL.

The CL-signature scheme defines the public key in this key pair as the quadruple $(n, Z, S ,\{R_i\}_{i \in A})$ where $A$ is the set of indices of attributes in the credential schema, $n$ is a Special RSA Modulus, and $Z, S , {R_i}$ are random quadratic residues modulo n.

The public key information $(n, Z, S ,\{R_i\}_{i \in A})$ are publicly available on the ledger, the signature $(A, e, v)$ can only be calculated by the issuer, who owns the private key $(p, q)$.

## Equations


Remember from the math section:

$$ T = A'^{\bar{e}} S^{\bar{v}} \Bigg(\prod_{i \in A_h}{R_i^{\bar{m_i}}} \Bigg) \pmod{n} $$
$$ A' = AS^r $$
$$ v' = v - er $$
$$ \hat{v} = \bar{v} + Dv'  $$
$$ e' = e - \beta $$
$$ \hat{e} = \bar{e} + De' $$
$$ \hat{m_i} = \bar{m_i} + Dm_i $$
$$ Q = A^e \pmod{n} $$
$$ Q = \Bigg(\frac{Z}{S^v \prod_{i \in A}{R_i^{m_i}}}\Bigg) \pmod{n} $$
$$ \hat{T} = \Bigg(\frac{Z}{A'^\beta \prod_{i \in A_r}{R_i^{m_i}}}\Bigg)^{-D} A'^{\hat{e}} S^{\hat{v}} \Bigg(\prod_{i \in A_h}{R_i^{\hat{m_i}}}\Bigg) \pmod{n} $$

## Issuer setup

Let $i$ be the number of attributes we work with.

Steps to set up the issuer

1. Issuer chooses a number of attribute bases $i$
2. Generate random primes of equal length ($p$ and $q$) 
3. Then compute the modulus $n \leftarrow pq$.
4. Generate a random quadratic residue: $S \pmod{n}$
5. Generate quadratic residues $(Z,\{R_i\}_{i \in A}) \pmod{n}$ 

The issuer's public key is $(n, S, Z, R_1,R_2,\ldots,R_i)$ and the private key is $(p, q)$.

In [1]:
from Crypto.Util import number
import hashlib
import math 


In [2]:
# For demo purposes to make print more readable and consistent
bitsize = 32
p = 3397833353
q = 2516557159
# p = number.getPrime(bitsize)
# q = number.getPrime(bitsize)

# The public parts
n = p*q
nsize = 2*bitsize
# Generate quadratic residues mod n
R_1 = pow(number.getRandomRange(2, n), 2, n)
R_2 = pow(number.getRandomRange(2, n), 2, n)
R_3 = pow(number.getRandomRange(2, n), 2, n)
R_4 = pow(number.getRandomRange(2, n), 2, n)
R_5 = pow(number.getRandomRange(2, n), 2, n)
R_6 = pow(number.getRandomRange(2, n), 2, n)
S = pow(number.getRandomRange(2, n), 2, n)
Z = pow(number.getRandomRange(2, n), 2, n)


print(f'Public key is:\nn = {n}\n(R_1, R_2, R_3, R_4, R_5, R_6) = {(R_1, R_2, R_3, R_4, R_5, R_6)}\nS = {S}\nZ = {Z}')
print(f'\nPrivate key is: {(p,q)}')

Public key is:
n = 8550841849581124127
(R_1, R_2, R_3, R_4, R_5, R_6) = (4740134512459538449, 355917158046286626, 3972404523570530078, 3346191599445879848, 4262799936457652554, 5775756261103085523)
S = 1166264759609768200
Z = 2781078383302462624

Private key is: (3397833353, 2516557159)


In [3]:
# The messages
msize = nsize
m_1 = number.getRandomRange(2, n)
m_2 = number.getRandomRange(2, n)
m_3 = number.getRandomRange(2, n)
m_4 = number.getRandomRange(2, n)
m_5 = number.getRandomRange(2, n)
m_6 = number.getRandomRange(2, n)

print(f'Message is:\nm_1, m_2, m_3, m_4, m_5, m_6)  = {(m_1, m_2, m_3, m_4, m_5, m_6)}')

Message is:
m_1, m_2, m_3, m_4, m_5, m_6)  = (4079624770403789574, 2621006226763439889, 7129322069437212076, 7359987611925054165, 7983619136427172893, 2268010931641300781)


## User

User loads Issuer's public key and calculate:

1. Generates random integer $v'$
2. Then computes
$$U \leftarrow S^{v'}R_1^{m_1}\pmod{n}$$

In [4]:
# Generate random integear
v_prim = number.getRandomNBitInteger(msize*nsize + 1)

# Calculate U
U = pow(S, v_prim, n) * pow(R_1, m_1, n)

### User send to Issuer
User send $(U, v')$ to the Issuer

## Issuer Signature Generation


Issuer generate CL signature on the attributes by computing:

1. Generates random integer $v''$
2. Generates random prime $e$
3. Then computes 
$$
Q \leftarrow \frac{Z}{U S^{v''}(R_2^{m_2}R_3^{m_3}\cdots  R_i^{m_i})}\pmod{n}.
$$
4. Then computes
$$
A \leftarrow Q^{e^{-1}\pmod{p'q'}}\pmod{n}.
$$


In [5]:
# Generate random prime
e = number.getPrime(nsize + 1)
e_inv = pow(e, -1, (p-1)*(q-1))
# v is selected as integer
v_pprim = number.getRandomNBitInteger(msize*nsize + 1)

# Calculate dot
R_vector = pow(R_2, m_2, n) * pow(R_3, m_3, n) * pow(R_4, m_4, n) * pow(R_5, m_5, n) * pow(R_6, m_6, n)

# Calculate commitment
commitment_vector = U * pow(S, v_pprim, n) * R_vector
commitment_vector_inv = pow(commitment_vector, -1, n)

# Calculate Q
Q = (Z * commitment_vector_inv) % n
# Calculate signature part A
A = pow(Q, e_inv, n)

print(f"Pre-signature is: A = {A}, e = {e}, v'' = {v_pprim}")

Pre-signature is: A = 7512016508607037149, e = 26160949446642253987, v'' = 1138290150733499066454047590477004107932589168760457250272474364934346238863259798832813929209464847977227393554020615069378720256035539161421524354733080729395411193655403289141570837793924914806476706128569375865818120174449277476231972775785724675217503269439489078714033337709521343902444695533064558409490677852885459694848408934609968244238502688217625431167091724232507657236485719518384090276013293145066773129927613238770430466103657000976879606149988238274545291158484948233759593671031918365074263954303840405067034735437452098193499626781091423650685462074371791970902990977647255973098413927548097652450566560154218571159725292701538415497536832423498011399758520630405902015524260153131430499875464759104620585406792960478892901936875807701420276973484641841999894090356981103273665102416013073773284626241801869063510413236357608981257439876438243225436609109382937598180372644754717186507725278856516332910620

Check
$$ Q = A^e $$

In [6]:
# verify
print('RHS is the same as LHS:', pow(A, e, n) == Q)

RHS is the same as LHS: True


### Issuer sends to User
Issuer sends pre-signature $(A,e,v'')$ to the User

## User complete signature

For signature completion user computes
$$v \leftarrow v'+v''$$

In [7]:
v = v_prim + v_pprim

User now know the signature $(A,e,v)$

User verify the signature $(A,e,v)$, using the CL verification algorithm:
1. Compute
$$
\hat{Z} \leftarrow A^{e} S^{v}(R_1^{m_1}R_2^{m_2}\cdots  R_i^{m_i})\pmod{n}.
$$

In [8]:
R_dot = pow(R_1, m_1, n) * R_vector
Z_hat = (pow(A, e, n) * pow(S, v, n) * R_dot) % n

Check
$$ Z = A^e S^v \Bigg(\prod_{i \in A}{R_i^{m_i}}\Bigg) $$

In [9]:
print('RHS is the same as LHS:', Z_hat == Z)

RHS is the same as LHS: True


## Proof Generation


### Selective disclosure
Let A be the set of all attribute identifiers present in the credential schema
such that $A_r$ contains revealed attributes to the Verifier and $A_h$ contains unrevealed (hidden) attributes which are kept secret.


In [10]:
# public attributes
R_public = pow(R_4, m_4, n) * pow(R_5, m_5, n)
# private attributes
R_private = pow(R_1, m_1, n) * pow(R_2, m_2, n) * pow(R_3, m_3, n) * pow(R_6, m_6, n)

We remember that:
$$ Z = A^e S^v \Bigg(\prod_{i \in A}{R_i^{m_i}}\Bigg) = 
A^e S^v \Bigg(\prod_{i \in A_r}{R_i^{m_i}} \cdot \prod_{i \in A_h}{R_i^{m_i}}\Bigg)
$$

Check
$$ Z \Bigg(\prod_{i \in A_r}{R_i^{m_i}}\Bigg)^{-1} = A^e S^v \Bigg(\prod_{i \in A_h}{R_i^{m_i}}\Bigg) $$

In [11]:
# generate commitment and encryption
R_public_inv = pow(R_public, -1, n)
# contains revealed attributes
attr_encryption_vector = Z * R_public_inv

# contains unrevealed attributes which are kept secret
attr_commitment_vector = pow(S, v, n) * R_private

# verify credentials
print('RHS is the same as LHS:', (pow(A, e, n) * attr_commitment_vector) % n == attr_encryption_vector % n)

RHS is the same as LHS: True


For each non-revealed attribute 

1. Generate random integer $\bar{m_1}, \bar{m_2}, \bar{m_3}$ for the secret attributes
2. Generates random integer $\bar{m_j}$ for each non-revealed attribute


In [12]:
m1_bar = number.getRandomNBitInteger(nsize)
m2_bar = number.getRandomNBitInteger(nsize)
m3_bar = number.getRandomNBitInteger(nsize)
m6_bar = number.getRandomNBitInteger(nsize)


Rbar_private = pow(R_1, m1_bar, n) * pow(R_2, m2_bar, n) * pow(R_3, m3_bar, n) * pow(R_6, m6_bar, n)


1. Generate random integer $r$
2. Then computes 
$$
A' \leftarrow A S^{r}\pmod{n}
$$
4. Then computes
$$
v' \leftarrow v - e\cdot r
$$


In [13]:
# Choose random bit r
r = number.getRandomNBitInteger(msize*nsize + 1)

# Calculate
A_prime = A * pow(S, r, n)
v_prime = v - e*r

We remember that:

$$
Z = A^e S^v \Bigg(\prod_{i \in A}{R_i^{m_i}}\Bigg) = A'^{e} S^{v'} \Bigg(\prod_{i \in A}{R_i^{m_i}}\Bigg) 
$$

Check
$$
A^e S^v = A'^{e} S^{v'}
$$
$$
A' S^{-r} = A \pmod{n}
$$

In [14]:
# we know that

print('RHS is the same as LHS:', (pow(A_prime, e, n) * pow(S, v_prime, n) % n * R_dot) % n == Z)
print('RHS is the same as LHS:', (pow(A_prime, e, n) * pow(S, v_prime, n) % n == pow(A, e, n) * pow(S, v, n) % n))
print('RHS is the same as LHS:', ((A_prime * pow(S, -r, n) % n == A % n)))

RHS is the same as LHS: True
RHS is the same as LHS: True
RHS is the same as LHS: True


1. Generate random integer $\beta$
2. Compute 
$$e' \leftarrow e - \beta$$

In [15]:
beta = number.getRandomNBitInteger(bitsize + 1)
e_prime = e - beta

print(f"Randomized signature is:\nA = {A_prime},\ne' = {e_prime},\nv' = {v_prime}")




Randomized signature is:
A = 31516415147843189870295925064287755522,
e' = 26160949440246648934,
v' = -3998229658140411470217871519963487575971011338473743489800283791974015710271907871070494965759537990182815472603582640739581773689926506347855338645365412345038912843908253977733895553369139005891546216590737647280925641058280318369567681329065151169940233280070979779782507900773591146809212439884048901483885524813984714524941313604028507002673873945775286156595179575165140221224700299897914444222869653221020130107383889481665299367850349645780016199508988707887571695683633342887859568936521061272257211637453394090680632626914666686595305120828307475922334928056808836931987850979359701420370927612161791202088475065917865260234956945682515108569725606730499352875941261859205450391058273306917642002385263893713676621071006818429245702672256192398972498636317290617666416450353951115015892523476689202948141693906668232778652610739435689229591790200093976672859653237917179760967699808364517

1. Generate random integer number $\bar{e}$
2. Generate random integer number $\bar{v}$
3. Compute
$$ T \leftarrow A'^{\bar{e}} S^{\bar{v}} \Bigg(\prod_{i \in A_h}{R_i^{\bar{m_i}}} \Bigg) \pmod{n} $$

In [16]:
e_bar = number.getRandomNBitInteger(nsize + 1)
v_bar = number.getRandomNBitInteger(msize*nsize+1)
T = (pow(A_prime, e_bar, n) * pow(S, v_bar, n) * Rbar_private) % n

### Hashing

Compute 
$$
D\leftarrow H(\mathcal{T},\mathcal{C},n_1)
$$

In [17]:
attr = str([T, Q, A, e, v, A_prime])
c_hash = hashlib.sha256(str(attr).encode('utf-8')).hexdigest()
c = int(c_hash, base=32)
print(f"Proof is: T = {T}, \nc = {c}")

Proof is: T = 5248475888255744825, 
c = 956346144591724637164394089938234809411346946177773091335727711534015596232501274355735678227904


Compute
\begin{align*}
\hat{v} \leftarrow \bar{v} + Dv' \\
\hat{e} \leftarrow \bar{e} + De'
\end{align*}

In [18]:
e_hat = e_bar + c*e_prime
v_hat = v_bar + c*v_prime

For each none revealed attribute, $j\in A_{h}$, compute
$$ \hat{m_j} \leftarrow \bar{m_j} + Dm_j $$


In [19]:
m1_hat = m1_bar + c*m_1
m2_hat = m2_bar + c*m_2
m3_hat = m3_bar + c*m_3
m6_hat = m6_bar + c*m_6

Rhat_private = pow(R_1, m1_hat, n) * pow(R_2, m2_hat, n) * pow(R_3, m3_hat, n) * pow(R_6, m6_hat, n)

We remember that:

$$

\Bigg({\prod_{i \in A_r}{R_i^{m_i}}}\Bigg)^{D} \cdot \Bigg(\prod_{i \in A_h}{R_i^{\hat{m_i}}}\Bigg)

= {\Bigg(\prod_{i \in A_r}{R_i^{Dm_i}}} \cdot \prod_{i \in A_h}{R_i^{Dm_i}}\Bigg) \cdot \prod_{i \in A_h}{R_i^{\bar{m_i}}}

= \Bigg(\prod_{i \in A}{R_i^{Dm_i}}\Bigg) \cdot \prod_{i \in A_h}{R_i^{\bar{m_i}}} \pmod{n}
$$

Lets check

$$

\Bigg({\prod_{i \in A_r}{R_i^{m_i}}}\Bigg)^{D} \cdot \Bigg(\prod_{i \in A_h}{R_i^{\hat{m_i}}}\Bigg)

= \Bigg(\prod_{i \in A}{R_i^{Dm_i}}\Bigg) \cdot \prod_{i \in A_h}{R_i^{\bar{m_i}}} \pmod{n}
$$


In [20]:
Rc_public = pow(R_4, m_4*c, n) * pow(R_5, m_5*c, n)
Rc_private = pow(R_1, m_1*c, n) * pow(R_2, m_2*c, n) * pow(R_3, m_3*c, n) * pow(R_6, m_6*c, n)
Rc_dot = Rc_private * Rc_public

print('RHS is the same as LHS:', (Rc_public * Rhat_private % n == Rc_dot * Rbar_private % n))

RHS is the same as LHS: True


## Proof Verification

Calculate

$$
\hat{T} \leftarrow \Bigg(\frac{Z}{A'^\beta \prod_{i \in A_r}{R_i^{m_i}}}\Bigg)^{-D} A'^{\hat{e}} S^{\hat{v}} \Bigg(\prod_{i \in A_h}{R_i^{\hat{m_i}}}\Bigg)
= Z^{-D} \cdot A'^{D\beta} \cdot A'^{\hat{e}} \cdot S^{\hat{v}} \cdot \Bigg({\prod_{i \in A_r}{R_i^{m_i}}}\Bigg)^{D} \cdot \Bigg(\prod_{i \in A_h}{R_i^{\hat{m_i}}}\Bigg) \pmod{n}
$$

In [21]:
T_hat = (pow(Z, -c, n) * pow(A_prime, c*beta, n) * pow(A_prime, e_hat, n) * pow(S, v_hat, n) * Rc_public * Rhat_private) % n

# we know that
print('RHS is the same as LHS:', (T_hat) == T)

RHS is the same as LHS: True


Compute
$$
\hat{Q} \leftarrow A'^{e} S^{-er} \pmod{n}
$$

In [22]:
Q_hat = pow(A_prime, e, n) * pow(S, -e*r, n) % n

Compute 
$$
\hat{D} \leftarrow H(\hat{\mathcal{T}},\mathcal{C},n_1);
$$

In [23]:


attr_hat = str([T_hat, Q_hat, A, e, v, A_prime])
c_hash_hat = hashlib.sha256(str(attr_hat).encode('utf-8')).hexdigest()
c_hat = int(c_hash_hat, base=32)

print('RHS is the same as LHS:', (c_hat) == c)

RHS is the same as LHS: True


## Equality proof
This zero knowledge proof is in fact just an equality proof that does not reveal the attribute’s value.

In [24]:
assert([T, c] == [T_hat, c_hat])

In [25]:
print('The equality has been VERIFIED when this line is printed!')

The equality has been VERIFIED when this line is printed!


# Predicate
Predicate has been broken out only to focus on it because it can itself be studied (almost) independently.


- https://dominoweb.draco.res.ibm.com/reports/rz3730_revised.pdf#page=26
- https://github.com/hyperledger/indy-hipe/blob/master/text/0109-anoncreds-protocol/README.md
- https://github.com/hyperledger-archives/indy-anoncreds/blob/master/docs/dev/anoncred.pdf

## Inequality predicates
Inequality predicates $\geq,  >, <, \leq, \neq$ are supported.
The holder takes the attribute $m_{j}$ from credential $\mathcal{C}$, and the term $a_i$ provided by the verifier. For each predicate ${p}$ computes the following:

1. Calculate ${\Delta}$ such that:

$$
\Delta \leftarrow \begin{cases}
a_i-m_{p_i}; & {if } \leq\\
a_i-m_{p_i}-1; & {if }  <\\
m_{p_i}-a_i; & {if } \geq or \neq\\
m_{p_i}-a_i-1; & {if } >
\end{cases}
$$

2. Calculate ${a}$ such that:

$$
b \leftarrow \begin{cases}
1; & {if } \geq, >, \neq\\
-1; & {if } \leq, <
\end{cases}
$$

3. Express ${\Delta}$ as the sum of four squares. Find (possibly by exhaustive search) $u_1, u_2, u_3, u_4$ such that:

$$
\Delta \leftarrow u_1^2 + u_2^2 +u_3^2 + u_4^2
$$

Note that ${\Delta}$ will always be non-negative if the predicate is true; cannot express a negative number as sum of four squares. If ${\Delta}$ < 0, the proof fails. 

#### Lagrange’s Four Square Theorem
Lagrange’s Four Square Theorem states that every natural number can be written as sum of squares of four non negative integers.

- [Lagrange's four-square theorem](https://en.wikipedia.org/wiki/Lagrange%27s_four-square_theorem)


##### How $\Delta$ is calculated
Lets check ${\Delta}$ for all inequality predicates such as "A is older than 20 years old".


Let age for A be 25 and calculate:
1. predict if age > 18
$$
\Delta \leftarrow 25 - 18 - 1 (= 6)
$$
2. predict if age < 18
$$
\Delta \leftarrow 18 - 25 - 1 (= -8)
$$

### ${\Delta}$ implementation

Implementation of ${\Delta}$ via
- https://github.com/evernym/sovrin-client-rust/blob/c3664cad7cd92c8cb661071c82fd3a1f4678a807/src/services/anoncreds/helpers.rs

Can later be implemented as:

- https://github.com/hyperledger/ursa/blob/5cd3331e1428daad73a0e0d857f8bd01affb4441/libursa/src/cl/helpers.rs#L472

In [26]:
def largest_square_less_than(delta):
    return int(math.floor(math.sqrt(float(delta))))


def four_squares(delta):
    d = int(delta)
    u1 = largest_square_less_than(d)
    u2 = largest_square_less_than(d - math.pow(u1, 2))
    u3 = largest_square_less_than(d - math.pow(u1, 2) - math.pow(u2, 2))
    u4 = largest_square_less_than(d - math.pow(u1, 2) - math.pow(u2, 2) - math.pow(u3, 2))

    uPow =  int(math.pow(u1, 2) + math.pow(u2, 2) + math.pow(u3, 2) + math.pow(u4, 2))
    if uPow == d:
        roots = [u1, u2, u3, u4]
    else:
        roots = [-1, -1, -1, -1]

    return roots

print(four_squares(22))
print(four_squares(4266287))

[4, 2, 1, 1]
[2065, 45, 6, 1]


Calculate $T$ equations

1. Generate random integer numbers $r_1, r_2, r_3, r_4, r_{\Delta}$
2. Compute 
$$ T_1 \leftarrow Z^{u_1}S^{r_1}\pmod{n} $$
$$ T_2 \leftarrow Z^{u_2}S^{r_2}\pmod{n} $$
$$ T_3 \leftarrow Z^{u_3}S^{r_3}\pmod{n} $$
$$ T_4 \leftarrow Z^{u_4}S^{r_4}\pmod{n} $$
$$ T_{\Delta} \leftarrow  Z^{\Delta}S^{r_{\Delta}} \pmod{n} $$

and add these values to $\mathcal{C}$ in the order ${T_1},{T_2},{T_3},{T_4},{T_{\Delta}}$.


In [27]:
# Generate random number
r1 = number.getRandomNBitInteger(msize*nsize + 1)
r2 = number.getRandomNBitInteger(msize*nsize + 1)
r3 = number.getRandomNBitInteger(msize*nsize + 1)
r4 = number.getRandomNBitInteger(msize*nsize + 1)
rd = number.getRandomNBitInteger(msize*nsize + 1)

# select m6 to do the predicate and compute delta
mp1 = m_6
mp1_bar = m6_bar
mp1_hat = m6_hat

ap1 = mp1 - 22
delta = mp1 - ap1
roots = four_squares(delta)
# check delta is greater than > 0
assert(sum(roots) > 0)
b = 1

# Compute T equations
T1 = pow(Z, roots[0], n) * pow(S, r1, n)
T2 = pow(Z, roots[1], n) * pow(S, r2, n)
T3 = pow(Z, roots[2], n) * pow(S, r3, n)
T4 = pow(Z, roots[3], n) * pow(S, r4, n)
Td = pow(Z, delta, n) * pow(S, rd, n)

print(f"Predicate is:\nDelta = {delta},\nRoots: (u_1, u_2, u_3, u_4) = {(roots[0], roots[1], roots[2], roots[3])}\nT_1 = {T1}\nT_2 = {T2}\nT_3 = {T3}\nT_4 = {T4}\nT_Delta = {Td}")


Predicate is:
Delta = 22,
Roots: (u_1, u_2, u_3, u_4) = (4, 2, 1, 1)
T_1 = 9986224797644691545539240195009981920
T_2 = 33455029329993013956431440036548620367
T_3 = 6323942909662065152866558236377245824
T_4 = 11324202009063269213864667726197521248
T_Delta = 2300667598011396800759621104221244750


Calculate $\bar{T}$ and $\bar{K}$ equations

1. Generate random integer numbers $\bar{u_1},\bar{u_2},\bar{u_3},\bar{u_4}$
2. Generate random integer numbers $\bar{r_1},\bar{r_2},\bar{r_3},\bar{r_4},\bar{r_{\Delta}}$
3. Compute 
$$ \bar{T_1} \leftarrow Z^{\bar{u_1}}S^{\bar{r_1}}\pmod{n} $$
$$ \bar{T_2} \leftarrow Z^{\bar{u_2}}S^{\bar{r_2}}\pmod{n} $$
$$ \bar{T_3} \leftarrow Z^{\bar{u_3}}S^{\bar{r_3}}\pmod{n} $$
$$ \bar{T_4} \leftarrow Z^{\bar{u_4}}S^{\bar{r_4}}\pmod{n} $$
$$ \bar{T_{\Delta}} \leftarrow  Z^{\bar{m_j}}(S^{b})^{\bar{r_{\Delta}}} \pmod{n} $$

and add this values to $\mathcal{T}$ in the order $\bar{T_1},\bar{T_2},\bar{T_3},\bar{T_4}, \bar{T_{\Delta}}$.

Where $\bar{m_j}$ is from the earlier generated random number.

4. Generate random integer numbers $\bar{\alpha}$
5. Compute
$$
\bar{K} \leftarrow S^{\bar{\alpha}}T_1^{\bar{u_1}}T_2^{\bar{u_2}}
T_3^{\bar{u_3}} T_4^{\bar{u_4}}\pmod{n}
$$
Add $\bar{K}$ to $\mathcal{T}$.

In [28]:
# Generate random number
u1_bar = number.getRandomNBitInteger(nsize + 1)
u2_bar = number.getRandomNBitInteger(nsize + 1)
u3_bar = number.getRandomNBitInteger(nsize + 1)
u4_bar = number.getRandomNBitInteger(nsize + 1)

r1_bar = number.getRandomNBitInteger(nsize + 1)
r2_bar = number.getRandomNBitInteger(nsize + 1)
r3_bar = number.getRandomNBitInteger(nsize + 1)
r4_bar = number.getRandomNBitInteger(nsize + 1)
rd_bar = number.getRandomNBitInteger(nsize + 1)

# Compute T_bar equations
T1_bar = pow(Z, u1_bar, n) * pow(S, r1_bar, n) % n
T2_bar = pow(Z, u2_bar, n) * pow(S, r2_bar, n) % n
T3_bar = pow(Z, u3_bar, n) * pow(S, r3_bar, n) % n
T4_bar = pow(Z, u4_bar, n) * pow(S, r4_bar, n) % n
Td_bar = pow(Z, mp1_bar, n) * pow(S, b*rd_bar, n) % n

# Compute K_bar
alpha_bar = number.getRandomNBitInteger(msize*nsize + 1)
K_bar = pow(S, alpha_bar, n) * pow(T1, u1_bar, n) * pow(T2, u2_bar, n) * pow(T3, u3_bar, n) * pow(T4, u4_bar, n) % n


print(f"Proof predicate is:\nRoots: (u1_bar, u2_bar, u3_bar, u4_bar) = {(u1_bar, u2_bar, u3_bar, u4_bar)}\nT1_bar = {T1_bar}\nT2_bar = {T2_bar}\nT3_bar = {T3_bar}\nT4_bar = {T4_bar}\nTd_bar = {Td_bar}\nalpha_bar = {alpha_bar}\nK_bar = {K_bar}")


Proof predicate is:
Roots: (u1_bar, u2_bar, u3_bar, u4_bar) = (29457504430408753849, 29860967272342552213, 28239681327794521258, 36253472686918861471)
T1_bar = 5476469918168233452
T2_bar = 7442350248140910299
T3_bar = 7840491588837811578
T4_bar = 7444455804813433209
Td_bar = 3128483255473031140
alpha_bar = 14548174708366998072802241205388268113573769325278967705058635971250316111034242515674234003021433232620719640720964828970008274791621213324063987385032615862258016634620645651378809984885569831465646937342297735176289838586180660102394099256800158462772704794125480428272867748805590199595969865288624159606079455040703911416863223038962671117382661581272111404933798960666258689089138487445912255507631421257202648102088512171878265980729676366429922454055813609569805871890079499963989888403920884374253857825481458994272500970147898382056369576041115881108941051233047057841414972407862344343010628074031819750007565073275201058186339207667443903668373648375802082076022820323558854397755

### Hashing

Compute 
$$
D\leftarrow H(\mathcal{T},\mathcal{C},n_1)
$$

In [29]:
# This is done in wrong order so will break the proof!!!

# attr = str([T, Q, T1_bar, T2_bar, T3_bar, T4_bar, Td_bar, K_bar, A, e, v, A_prime, T1, T2, T3, T4, Td])
# c_hash = hashlib.sha256(str(attr).encode('utf-8')).hexdigest()
# c = int(c_hash, base=32)
# print(f"c = {c}")

### Holder generate predicate

For each predicate $p$ compute:

1. For {$i \in {1, 2, 3, 4}$}, compute
\begin{align*}
\hat{u_i} \leftarrow \bar{u_i}+Du_i \\
\hat{r_i} \leftarrow \bar{r_i}+Dr_i \\
\end{align*}

2. Compute
$$
\hat{r_{\Delta}} \leftarrow \bar{r_{\Delta}}+Dr_{\Delta}
$$

3. Compute
$$
{\alpha} \leftarrow r_{\Delta}- u_1r_1 - u_2r_2 - u_3r_3 - u_4r_4
$$

4. Compute
$$
\hat{\alpha} \leftarrow \bar{\alpha}+D\alpha
$$


In [30]:
u1_hat = u1_bar + c*roots[0]
u2_hat = u2_bar + c*roots[1]
u3_hat = u3_bar + c*roots[2]
u4_hat = u4_bar + c*roots[3]

r1_hat = r1_bar + c*r1
r2_hat = r2_bar + c*r2
r3_hat = r3_bar + c*r3
r4_hat = r4_bar + c*r4

rd_hat = rd_bar + c*rd

alpha = rd - roots[0]*r1 - roots[1]*r2 - roots[2]*r3 - roots[3]*r4
alpha_hat = alpha_bar + c*alpha

# verify
assert(pow(T1, u1_hat, n) == pow(T1, u1_bar, n) * pow(T1, c*roots[0], n) % n)
assert(pow(T2, u2_hat, n) == pow(T2, u2_bar, n) * pow(T2, c*roots[1], n) % n)
assert(pow(T3, u3_hat, n) == pow(T3, u3_bar, n) * pow(T3, c*roots[2], n) % n)
assert(pow(T4, u4_hat, n) == pow(T4, u4_bar, n) * pow(T4, c*roots[3], n) % n)

The values $Pr_p =( \{\hat{u_i}\}, \{\hat{r_i}\},\hat{r_{\Delta}},\hat{\alpha},\hat{m_j})$ are the sub-proof for predicate $p$.

The values $Pr_C=(\hat{e},\hat{v},\hat{m_j},A')$ are the sub-proof for credential $C$.

Then $(D,\{Pr_C\},\{Pr_p\},\mathcal{C})$ is the full proof sent to the Verifier.

## Verifier validate predicate
### Proof generation

For each predicate $p$

1. Calculate ${\Delta}'$ such that:

$$
\Delta' \leftarrow \begin{cases}
a_i; & {if } \leq\\
a_i-1; & {if }  <\\
a_i; & {if } \geq or \neq\\
a_i+1; & {if } >
\end{cases}
$$

2. Calculate ${b}$ such that:

$$
b \leftarrow \begin{cases}
1; & {if } \geq, >, \neq\\
-1; & {if } \leq, <
\end{cases}
$$

Note that 
$$
\Delta' \leftarrow m_{p_i} - b\Delta
$$

Show
$$
\Delta' \leftarrow \begin{cases}
m - b(a-m) = m + (a - m) = a; & {if } \leq\\
m - b(a-m-1) = m + (a-m-1) =  a - 1; & {if }  <\\
m - b(m-a) = m - (m-a) = a; & {if } \geq or \neq\\
m - b(m-a-1) = m - (m-a-1) = a+1; & {if } >
\end{cases}
$$


### Proof verification

Using $Pr_p$ and $\mathcal{C}$ compute 

1. $\text{For }1\leq i \leq 4$, compute
$$
\hat{T_i} \leftarrow T_i^{-D}Z^{\hat{u_i}} S^{\hat{r_i}}\pmod{n};
$$

2. Compute
$$
\hat{T_{\Delta}} \leftarrow \left((T_{\Delta})^bZ^{\Delta'}\right)^{-D}Z^{\hat{m_j}}(S^b)^{\hat{r_{\Delta}}}\pmod{n};
$$

3. Compute
$$
\hat{K}\leftarrow S^{\hat{\alpha}}(T_{\Delta})^{-D}T_1^{\hat{u_1}}T_2^{\hat{u_2}}T_3^{\hat{u_3}}T_4^{\hat{u_4}}
\pmod{n},
$$
and add these values to  $\hat{\mathcal{T}}$ in the order $\hat{T_1},\hat{T_2} ,\hat{T_3},\hat{T_4},\hat{T_{\Delta}},\hat{K}    $.

In [31]:
delta_prim = ap1

# Compute T_hat equations
T1_hat = pow(T1, -c, n) * pow(Z, u1_hat, n) * pow(S, r1_hat, n) % n
T2_hat = pow(T2, -c, n) * pow(Z, u2_hat, n) * pow(S, r2_hat, n) % n
T3_hat = pow(T3, -c, n) * pow(Z, u3_hat, n) * pow(S, r3_hat, n) % n
T4_hat = pow(T4, -c, n) * pow(Z, u4_hat, n) * pow(S, r4_hat, n) % n
Td_hat = pow(Td, -b*c, n) * pow(Z, -delta_prim*c, n) *  pow(Z, mp1_hat, n) * pow(S, b * rd_hat, n)  % n


# Compute K_hat
K_hat = pow(S, alpha_hat, n) * pow(Td, -c, n) * pow(T1, u1_hat, n) * pow(T2, u2_hat, n) * pow(T3, u3_hat, n) * pow(T4, u4_hat, n) % n

# add values to tau
tau = [T1_bar, T2_bar, T3_bar, T4_bar, Td_bar, K_bar]
tau_hat = [T1_hat, T2_hat, T3_hat, T4_hat, Td_hat, K_hat]

Compute 
$$
\hat{D} \leftarrow H(\hat{\mathcal{T}},\mathcal{C},n_1);
$$
If $D=\hat{D}$ output VERIFIED else FAIL.

In [32]:
# build hash array
C_m = [A, e, v, A_prime]
C_t = [T1, T2, T3, T4, Td]
tau1_m = [T, Q]
tau1_p = [T1_bar, T2_bar, T3_bar, T4_bar, Td_bar, K_bar]
tau2_m = [T_hat, Q_hat]
tau2_p = [T1_hat, T2_hat, T3_hat, T4_hat, Td_hat, K_hat]

C = [C_m, C_t]
tau = [tau1_m, tau1_p]
tau_hat = [tau2_m, tau2_p]

# hashing
attr = str([tau, C])
c_hash = hashlib.sha256(str(attr).encode('utf-8')).hexdigest()
c = int(c_hash, base=32)

attr_hat = str([tau_hat, C])
c_hash_hat = hashlib.sha256(str(attr_hat).encode('utf-8')).hexdigest()
c_hat = int(c_hash_hat, base=32)

print('RHS is the same as LHS:', (c_hat) == c)

RHS is the same as LHS: True


In [33]:
# Help for extra verification
assert(T1_hat == T1_bar)
assert(T2_hat == T2_bar)
assert(T3_hat == T3_bar)
assert(T4_hat == T4_bar)
assert(Td_hat == Td_bar)
assert(K_hat == K_bar)

In [ ]:
# check Td_hat
assert(-c*mp1 == -b*c*delta - c*delta_prim)
assert(mp1_bar == -b*c*delta - c*delta_prim + mp1_hat)

In [34]:
# Check equality
assert([T, c] == [T_hat, c_hat])

In [35]:
print('The equality has been VERIFIED when this line is printed!')

The equality has been VERIFIED when this line is printed!
